In [9]:
# pytorch mlp for regression
from numpy import vstack
import numpy as np
from numpy import sqrt
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import Tanh
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.optim import Adam
from torch.nn import MSELoss
from torch.nn.init import xavier_uniform_
 

# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.values[:, :-2].astype('float32')
        self.y = df.values[:, -2:].astype('float32')
        # print(len(self.X[0]))
        # ensure target has the right shape
        self.y = self.y.reshape((len(self.y), 2))
        
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    # get indexes for train and test rows
    def get_splits(self, n_test=0.20):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])
 
# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs,layer1, layer2, act_func):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, layer1)
        xavier_uniform_(self.hidden1.weight)
        self.act1 = act_func()
        # second hidden layer
        self.hidden2 = Linear(layer1, layer2)
        xavier_uniform_(self.hidden2.weight)
        self.act2 = act_func()
        # third hidden layer and output
        self.hidden3 = Linear(layer2, 2)
        xavier_uniform_(self.hidden3.weight)
 
    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        return X
 
# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    print(CSVDataset(path).__getitem__(0))
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl
 
# train the model
def train_model(train_dl, model, learn):
    # define the optimization
    criterion = MSELoss()
    optimizer = Adam(model.parameters(), lr=learn)
    # enumerate epochs
    for epoch in range(1000):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
 
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 2))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate mse
    mse = mean_squared_error(actuals, predictions)
    mae = mean_absolute_error(actuals, predictions)
    return mse, mae
 
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat
 
# prepare the data
path = 'DataSet2.csv'
train_dl, test_dl = prepare_data(path)
# print(len(train_dl.dataset), len(test_dl.dataset))

[array([-59., -56., -57., -52., -51., -59., -62., -57., -60., -58., -63.,
       -66., -58., -62., -62.], dtype=float32), array([2., 2.], dtype=float32)]


In [14]:
with open('results4.csv', 'a') as f:
    f.write('layer1,layer2,actFunc,learning,mse,mae\n') 
layers1 = [10,30,50,70,90,110]
layers2 = [10,30,50,70,90,110]
actFunc = [ReLU,Sigmoid,Tanh]
learning = [0.1,0.03,0.01,0.003,0.001,0.0001]
for a in layers1:
    for b in layers2:
        for c in actFunc:
            for d in learning:
                # define the network
                
                model = MLP(15,a, b, c)
                # train the model
                train_model(train_dl, model,d)
                # evaluate the model
                mse, mae = evaluate_model(test_dl, model)
                with open('results4.csv', 'a') as f:
                    f.write(f'{a},{b},{c},{d},{mse},{mae}\n')
                print('MSE: %.3f, RMSE: %.3f' % (mse, sqrt(mse)))
                print('MAE: %.3f' % (mae))

                # make a single prediction (expect class=1)
                row = [-56,-54,-60,-54,-58,-60,-55,-59,-59,-54,-54,-56,-59,-52,-56]
                yhat = predict(row, model)
                print(f'Predicted: {yhat}')

MSE: 9.503, RMSE: 3.083
MAE: 2.544
Predicted: [[5.838864 6.026923]]
MSE: 3.633, RMSE: 1.906
MAE: 1.528
Predicted: [[3.866815  7.8123384]]
MSE: 3.317, RMSE: 1.821
MAE: 1.307
Predicted: [[2.933699 4.408281]]
MSE: 1.322, RMSE: 1.150
MAE: 0.830
Predicted: [[3.945289 5.622331]]
MSE: 1.889, RMSE: 1.374
MAE: 1.027
Predicted: [[4.0504184 4.971695 ]]
MSE: 4.185, RMSE: 2.046
MAE: 1.743
Predicted: [[4.635137  4.6741276]]
MSE: 9.725, RMSE: 3.119
MAE: 2.712
Predicted: [[5.112684 5.692278]]
MSE: 9.639, RMSE: 3.105
MAE: 2.582
Predicted: [[5.939592 5.710804]]
MSE: 9.537, RMSE: 3.088
MAE: 2.589
Predicted: [[5.7235966 5.8388686]]
MSE: 9.493, RMSE: 3.081
MAE: 2.553
Predicted: [[5.752366 5.999447]]
MSE: 9.498, RMSE: 3.082
MAE: 2.576
Predicted: [[5.668573 5.949016]]
MSE: 4.241, RMSE: 2.059
MAE: 1.746
Predicted: [[4.2875705 6.976698 ]]
MSE: 9.571, RMSE: 3.094
MAE: 2.564
Predicted: [[5.893961 5.835129]]
MSE: 9.486, RMSE: 3.080
MAE: 2.571
Predicted: [[5.8392477 6.213927 ]]
MSE: 9.625, RMSE: 3.102
MAE: 2.557
P